In [ ]:
# Bài 1
from simpleai.search import SearchProblem, astar
import time

GOAL_STATE = 'ABCDEFGH'

# --- Problem with heuristic H1 ---
class ProblemH1(SearchProblem):
    def actions(self, state):
        n = len(state)
        action_list = []
        for i in range(n):
            for j in range(i + 1, n):
                action_list.append((i, j))
        return action_list

    def result(self, state, action):
        i, j = action
        state_list = list(state)
        temp = state_list[i]
        state_list[i] = state_list[j]
        state_list[j] = temp
        return ''.join(state_list)

    def is_goal(self, state):
        if state == GOAL_STATE:
            return True
        else:
            return False

    def heuristic(self, state):
        mismatch_count = 0
        for k in range(len(GOAL_STATE)):
            if state[k] != GOAL_STATE[k]:
                mismatch_count += 1
        return mismatch_count

# --- Problem with heuristic H2 ---
class ProblemH2(SearchProblem):
    def actions(self, state):
        n = len(state)
        action_list = []
        for i in range(n):
            for j in range(i + 1, n):
                action_list.append((i, j))
        return action_list

    def result(self, state, action):
        i, j = action
        state_list = list(state)
        temp = state_list[i]
        state_list[i] = state_list[j]
        state_list[j] = temp
        return ''.join(state_list)

    def is_goal(self, state):
        if state == GOAL_STATE:
            return True
        else:
            return False

    def heuristic(self, state):
        misplaced = 0
        for k in range(len(GOAL_STATE)):
            if state[k] != GOAL_STATE[k]:
                misplaced += 1

        not_in_goal = 0
        for letter in GOAL_STATE:
            found = False
            for s in state:
                if s == letter:
                    found = True
                    break
            if not found:
                not_in_goal += 1

        return misplaced + not_in_goal

# --- Main execution ---
if __name__ == "__main__":
    start_state = 'HGFEDCBA'

    # Run A* with H1
    start_time = time.time()
    problem_h1 = ProblemH1(start_state)
    result_h1 = astar(problem_h1, graph_search=False)
    time_h1 = time.time() - start_time

    # Run A* with H2
    start_time = time.time()
    problem_h2 = ProblemH2(start_state)
    result_h2 = astar(problem_h2, graph_search=False)
    time_h2 = time.time() - start_time

    print("=== H1 (misplaced letters) ===")
    print("Path:", result_h1.path())
    print("Number of steps:", len(result_h1.path()))
    print("Execution time:", time_h1)

    print("\n=== H2 (misplaced + missing letters) ===")
    print("Path:", result_h2.path())
    print("Number of steps:", len(result_h2.path()))
    print("Execution time:", time_h2)


=== H1 (misplaced letters) ===
Path: [(None, 'HGFEDCBA'), ((0, 7), 'AGFEDCBH'), ((1, 6), 'ABFEDCGH'), ((2, 5), 'ABCEDFGH'), ((3, 4), 'ABCDEFGH')]
Number of steps: 5
Execution time: 0.0004093647003173828

=== H2 (misplaced + missing letters) ===
Path: [(None, 'HGFEDCBA'), ((0, 7), 'AGFEDCBH'), ((1, 6), 'ABFEDCGH'), ((2, 5), 'ABCEDFGH'), ((3, 4), 'ABCDEFGH')]
Number of steps: 5
Execution time: 0.0005741119384765625


In [6]:
# Bài 2
from simpleai.search import SearchProblem
from simpleai.search.local import genetic
import random

N = 8

class NQueensProblem(SearchProblem):
    def generate_random_state(self):
        # Mỗi phần tử là vị trí hàng của hậu ở cột i
        state = []
        for _ in range(N):
            row = random.randint(0, N - 1)
            state.append(row)
        return tuple(state)

    def crossover(self, state1, state2):
        cut_point = random.randint(1, N - 1)
        child = []
        # lấy từ state1
        for i in range(cut_point):
            child.append(state1[i])
        # lấy phần còn lại từ state2
        for i in range(cut_point, N):
            child.append(state2[i])
        return tuple(child)

    def mutate(self, state):
        index = random.randint(0, N - 1)
        new_row = random.randint(0, N - 1)
        state_list = []
        for i in range(N):
            if i == index:
                state_list.append(new_row)
            else:
                state_list.append(state[i])
        return tuple(state_list)

    def value(self, state):
        # Đếm số cặp hậu không tấn công nhau
        non_attacking = 0
        for i in range(N):
            for j in range(i + 1, N):
                same_row = (state[i] == state[j])
                same_diag = (abs(state[i] - state[j]) == j - i)
                if not same_row and not same_diag:
                    non_attacking += 1
        return non_attacking

problem = NQueensProblem()

result = genetic(problem,
                 population_size=100,
                 mutation_chance=0.05,
                 iterations_limit=1000)

print("Best solution found:", result.state)
print("Fitness value:", problem.value(result.state))

Best solution found: (4, 7, 3, 0, 2, 5, 1, 6)
Fitness value: 28


In [5]:
from easyAI.TwoPlayerGame import TwoPlayerGame
from easyAI import Human_Player, AI_Player, Negamax

RED = '\033[91m'
GREEN = '\033[92m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
RESET = '\033[0m'

WINNING_COMBOS = [
    [1,2,3],[4,5,6],[7,8,9],
    [1,4,7],[2,5,8],[3,6,9],
    [1,5,9],[3,5,7]
]

class GameController(TwoPlayerGame):
    def __init__(self, players):
        self.players = players
        self.current_player = 1  # player 1 bắt đầu
        self.board = [0] * 9

    def possible_moves(self):
        moves = []
        index = 0
        for value in self.board:
            if value == 0:
                moves.append(index + 1)  # vị trí đánh là 1..9
            index += 1
        return moves

    def make_move(self, move):
        idx = int(move) - 1
        self.board[idx] = self.nplayer

    def loss_condition(self):
        opponent = 3 - self.current_player  # giữ nguyên cách bạn đang dùng
        for combo in WINNING_COMBOS:
            win = True
            for cell in combo:
                if self.board[cell - 1] != opponent:
                    win = False
                    break
            if win:
                return True
        return False

    def is_over(self):
        no_moves_left = False
        if len(self.possible_moves()) == 0:
            no_moves_left = True

        if no_moves_left or self.loss_condition():
            return True
        else:
            return False

    def show(self):
        print(f"\n{BLUE}=== TIC-TAC-TOE ==={RESET}")
        print(f"{YELLOW}Vị trí bảng:{RESET}")
        print("1 2 3\n4 5 6\n7 8 9")
        print(f"{YELLOW}Trạng thái hiện tại:{RESET}")

        row = 0
        while row < 3:
            col = 0
            symbols = []
            while col < 3:
                idx = 3 * row + col
                value = self.board[idx]
                if value == 0:
                    symbols.append(str(idx + 1))
                elif value == 1:
                    symbols.append(GREEN + 'O' + RESET)
                else:
                    symbols.append(RED + 'X' + RESET)
                col += 1
            print(' '.join(symbols))
            row += 1
        print()

    def scoring(self):
        if self.loss_condition():
            return -100
        else:
            return 0

    def announce_winner(self):
        print(f"\n{BLUE}=== KẾT QUẢ GAME ==={RESET}")
        if self.loss_condition():
            winner = 3 - self.current_player
            if winner == 1:
                print(f"{GREEN} Player 1 (O) THẮNG! {RESET}")
            else:
                print(f"{RED} Player 2 (X) THẮNG! {RESET}")
        else:
            print(f"{YELLOW} GAME HÒA! {RESET}")

In [ ]:
# Bài 3
from simpleai.search import SearchProblem
from simpleai.search.local import simulated_annealing, hill_climbing
import random

N = 8

class NQueensProblem(SearchProblem):
    def __init__(self):
        initial_state = [random.randint(0, N-1) for _ in range(N)]
        super().__init__(initial_state)

    def actions(self, state):
        actions = []
        for row in range(N):
            for col in range(N):
                if col != state[row]:
                    actions.append((row, col))
        return actions

    def result(self, state, action):
        row, col = action
        new_state = list(state)
        new_state[row] = col
        return new_state

    def value(self, state):
        non_attacking = 0
        for i in range(N):
            for j in range(i+1, N):
                if state[i] != state[j] and abs(state[i]-state[j]) != j-i:
                    non_attacking += 1
        return non_attacking

# --- Hàm hiển thị bàn cờ ---
def print_board(state):
    for row in range(N):
        line = ['Q' if state[row]==col else '.' for col in range(N)]
        print(' '.join(line))
    print()

best_sa = None
best_hc = None
best_sa_value = -1
best_hc_value = -1

for i in range(10):
    problem = NQueensProblem()
    
    sa_result = simulated_annealing(problem, iterations_limit=1000)
    hc_result = hill_climbing(problem, iterations_limit=1000)

    sa_val = problem.value(sa_result.state)
    hc_val = problem.value(hc_result.state)

    if sa_val > best_sa_value:
        best_sa_value = sa_val
        best_sa = sa_result.state

    if hc_val > best_hc_value:
        best_hc_value = hc_val
        best_hc = hc_result.state

print("=== Kết quả tốt nhất Simulated Annealing ===")
print_board(best_sa)
print("Fitness:", best_sa_value)

print("=== Kết quả tốt nhất Hill Climbing ===")
print_board(best_hc)
print("Fitness:", best_hc_value)

=== Kết quả tốt nhất Simulated Annealing ===
. . . Q . . . .
. Q . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
Q . . . . . . .
. . Q . . . . .
. . . . . . Q .

Fitness: 28
=== Kết quả tốt nhất Hill Climbing ===
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . Q . .
. . . . . . . Q
. Q . . . . . .
. . . . Q . . .

Fitness: 27
